In [24]:
#installed jupyter lab and created a virtue environment
#for instructions go to https://janakiev.com/blog/jupyter-virtual-envs/
#thought i needed to used google big query to use the data but it seems like I am able to use their web browser at export a csv

#importing relevant packages/modules
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sklearn as sk
import pylab as pl

#setting option for easier dataframe viewing, can write None if we don't want a max
pd.set_option("display.max_rows", 400, "display.max_columns", 160)

In [ ]:
#next we need to scrape data from sports reference
#saw a reddit post that they have measures in place to prevent excessive scraping so this may or may not work

In [6]:
#real cell

#this is the cell for scraping data for school stats

#this is the creation for a single year with basic school stats
#this variable will need to be changed at the begining of running for a new year
#skipping for 2020 though because there was no tournoment, will likely need to adjust the code in the future to reflect this
current_year = 2019
# URL page we will scraping
url = "https://www.sports-reference.com/cbb/seasons/"+ str(current_year) +"-school-stats.html"
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll('tr', limit=2)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
# NCAA season we will be analyzing

#cleaning up header names
length = len(headers)
for i in range(length):
    if i == 8 or i == 9:
        headers[i] = 'Conf'+headers[i]
    elif i == 11 or i == 12:
        headers[i] = 'Home'+headers[i]
    elif i == 14 or i == 15:
        headers[i] = 'Away'+headers[i]

# avoid the first header row
rows = soup.findAll('tr')[2:]
currentpage_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

#ceating dataframe
stats = pd.DataFrame(currentpage_stats, columns = headers)

#removing missing columns and rows
stats = stats.dropna(subset= ['School'])
stats = stats.drop(['\xa0'], axis=1)


#replicating with advanced stats table

#the type of table we'll look at, there are multiple tables on a page, although depending on the year there may be no tables for opposition or advanced stats
TableType = ['-advanced-school', '-opponent', '-advanced-opponent']

for tt in TableType:
    #url stuff
    url = "https://www.sports-reference.com/cbb/seasons/"+ str(current_year) + tt +"-stats.html"
    html = urlopen(url)
    soup = BeautifulSoup(html)
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    headers = headers[1:]
    
    #changing headers depending on table
    if tt == '-opponent' or tt == '-advanced-opponent':
        j = 0
        for i in headers:
            if i !=  'School':
                headers[j] = 'opp'+ i
            j = j + 1    
    
    rows = soup.findAll('tr')[2:]
    currentpage_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

    #ceating dataframe
    Astats = pd.DataFrame(currentpage_stats, columns = headers)

    #removing depending on column type
    if tt == '-opponent' or tt == '-advanced-opponent':
        #removing missing columns and rows and excess columns
        Astats = Astats.dropna(subset= ['School'])
        Astats = Astats.drop(['oppG','oppW','oppL','oppW-L%','oppSRS','oppSOS','oppW','oppL','oppW','oppL','oppW','oppL','oppTm.','oppOpp.','opp\xa0'], axis=1)
    else:
        Astats = Astats.dropna(subset= ['School'])
        Astats = Astats.drop(['G','W','L','W-L%','SRS','SOS','W','L','W','L','W','L','Tm.','Opp.','\xa0'], axis=1)

    #combining this table with the previous table 
    stats = stats.merge(Astats, how = 'outer', on = 'School')

#add the year to the column
stats['year'] = current_year

#replicating previous table work and looping through years before appending, data for individual teams only goes back to 1993, for opp only goes back to 2010
#if model package cant deal with missing data will need to either not bring in opp data or will need to fill in missing 

year = list(range(1993,current_year))

for yr in year:
    url = "https://www.sports-reference.com/cbb/seasons/"+ str(yr) +"-school-stats.html"
    html = urlopen(url)
    soup = BeautifulSoup(html)
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    headers = headers[1:]
    length = len(headers)
    length = len(headers)
    length = len(headers)
    for i in range(length):
        if i == 8 or i == 9:
            headers[i] = 'Conf'+headers[i]
        elif i == 11 or i == 12:
            headers[i] = 'Home'+headers[i]
        elif i == 14 or i == 15:
            headers[i] = 'Away'+headers[i]
    rows = soup.findAll('tr')[2:]
    currentpage_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    pstats = pd.DataFrame(currentpage_stats, columns = headers)
    pstats = pstats.dropna(subset= ['School'])
    pstats = pstats.drop(['\xa0'], axis=1)
    
    #creat exception for curtian years where all the data is not preasent 
    if yr < 2010:
        TableType = ['-advanced-school']
    else:
        TableType = ['-advanced-school', '-opponent', '-advanced-opponent']
    
    for tt in TableType:
        url = "https://www.sports-reference.com/cbb/seasons/"+ str(yr) + tt +"-stats.html"
        html = urlopen(url)
        soup = BeautifulSoup(html)
        soup.findAll('tr', limit=2)
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
        headers = headers[1:]
        if tt == '-opponent' or tt == '-advanced-opponent':
            j = 0
            for i in headers:
                if i !=  'School':
                    headers[j] = 'opp'+ i
                j = j + 1    
        rows = soup.findAll('tr')[2:]
        currentpage_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
        pAstats = pd.DataFrame(currentpage_stats, columns = headers)
        if tt == '-opponent' or tt == '-advanced-opponent':
            pAstats = pAstats.dropna(subset= ['School'])
            pAstats = pAstats.drop(['oppG','oppW','oppL','oppW-L%','oppSRS','oppSOS','oppW','oppL','oppW','oppL','oppW','oppL','oppTm.','oppOpp.','opp\xa0'], axis=1)
        else:
            pAstats = pAstats.dropna(subset= ['School'])
            pAstats = pAstats.drop(['G','W','L','W-L%','SRS','SOS','W','L','W','L','W','L','Tm.','Opp.','\xa0'], axis=1)   
        pstats = pstats.merge(pAstats, how = 'outer', on = 'School')
    
    # issues with merging when number of columns between merging data frame and if colomns are repeated, this line adds columns to fix issue
    #header_list = 'School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'Conf ', 'ConfW', 'HomeL', 'Home ', 'AwayW', 'AwayL', 'W', 'L', 'Tm.', 'Opp.', 'MP', 'FG','FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB','AST', 'STL', 'BLK', 'TOV', 'PF', 'Pace', 'ORtg', 'FTr', '3PAr', 'TS%','TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA','oppMP', 'oppFG', 'oppFGA', 'oppFG%', 'opp3P', 'opp3PA', 'opp3P%','oppFT', 'oppFTA', 'oppFT%', 'oppORB', 'oppTRB', 'oppAST', 'oppSTL','oppBLK', 'oppTOV', 'oppPF', 'oppPace', 'oppORtg', 'oppFTr', 'opp3PAr','oppTS%', 'oppTRB%', 'oppAST%', 'oppSTL%', 'oppBLK%', 'oppeFG%','oppTOV%', 'oppORB%', 'oppFT/FGA'
   # if len(pstats.columns) == 47:
      #  pstats = pstats.reindex(columns = header_list)
    
    pstats['year'] = yr
    #appending stats table 
    stats = stats.append(pstats) 
#removing unwanted characters form School names
stats['School'] = stats['School'].map(lambda x: x.rstrip('\xa0NCAA'))

In [62]:
#test cell

pstats.columns
#len(pstats.columns)

Index(['School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'ConfW', 'ConfL',
       'Home ', 'HomeW', 'AwayL', 'Away ', 'W', 'L', 'Tm.', 'Opp.', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'Pace', 'ORtg', 'FTr', '3PAr', 'TS%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
       'year'],
      dtype='object')

In [63]:
#test cell
stats.columns

Index(['School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'ConfW', 'ConfL',
       'Home ', 'HomeW', 'AwayL', 'Away ', 'W', 'L', 'Tm.', 'Opp.', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'Pace', 'ORtg', 'FTr', '3PAr', 'TS%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
       'oppMP', 'oppFG', 'oppFGA', 'oppFG%', 'opp3P', 'opp3PA', 'opp3P%',
       'oppFT', 'oppFTA', 'oppFT%', 'oppORB', 'oppTRB', 'oppAST', 'oppSTL',
       'oppBLK', 'oppTOV', 'oppPF', 'oppPace', 'oppORtg', 'oppFTr', 'opp3PAr',
       'oppTS%', 'oppTRB%', 'oppAST%', 'oppSTL%', 'oppBLK%', 'oppeFG%',
       'oppTOV%', 'oppORB%', 'oppFT/FGA', 'year'],
      dtype='object')

In [4]:
#test cell
#df1 = pd.DataFrame({"A": [1,2,3], "B": [2,3,4]})

#df1 = df1.reindex(columns = df1.columns.tolist() + ['oppMP', 'oppFG', 'oppFGA', 'oppFG%', 'opp3P', 'opp3PA', 'opp3P%',
 #      'oppFT', 'oppFTA', 'oppFT%', 'oppORB', 'oppTRB', 'oppAST', 'oppSTL',
  #     'oppBLK', 'oppTOV', 'oppPF', 'oppPace', 'oppORtg', 'oppFTr', 'opp3PAr',
  #     'oppTS%', 'oppTRB%', 'oppAST%', 'oppSTL%', 'oppBLK%', 'oppeFG%',
  #    'oppTOV%', 'oppORB%', 'oppFT/FGA'])


stats

,School,G,W,L,W-L%,SRS,SOS,ConfW,ConfL,HomeW,HomeL,AwayW,AwayL,Tm.,Opp.,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,oppMP,oppFG,oppFGA,oppFG%,opp3P,opp3PA,opp3P%,oppFT,oppFTA,oppFT%,oppORB,oppTRB,oppAST,oppSTL,oppBLK,oppTOV,oppPF,oppPace,oppORtg,oppFTr,opp3PAr,oppTS%,oppTRB%,oppAST%,oppSTL%,oppBLK%,oppeFG%,oppTOV%,oppORB%,oppFT/FGA,year
0,Abilene Christian,34,27,7,.794,-1.91,-7.34,14,4,13,2,10,4,2502,2161,1370,897,1911,.469,251,660,.380,457,642,.712,325,1110,525,297,93,407,635,67.2,108.6,.336,.345,.565,50.3,58.5,12.9,8.0,.535,15.5,28.8,.239,1370,753,1737,.434,191,578,.330,464,643,.722,295,1098,356,192,92,560,592,67.2,93.8,.370,.333,.529,49.7,47.3,8.3,7.4,.488,21.5,27.3,.267,2019
1,Air Force,32,14,18,.438,-4.28,0.24,8,10,9,6,3,9,2179,2294,1300,802,1776,.452,234,711,.329,341,503,.678,253,1077,434,154,57,423,543,67.4,99.5,.283,.400,.541,50.1,54.1,7.0,5.8,.517,17.4,23.7,.192,1300,821,1820,.451,299,841,.356,353,520,.679,258,1073,443,231,87,386,550,67.4,104.7,.286,.462,.555,49.9,54.0,10.5,8.2,.533,15.7,23.8,.194,2019
2,Akron,33,17,16,.515,4.86,1.09,8,10,14,3,1,10,2271,2107,1325,797,1948,.409,297,929,.320,380,539,.705,312,1204,399,185,106,388,569,68.5,100.1,.277,.477,.515,48.2,50.1,8.2,8.9,.485,15.0,25.3,.195,1325,760,1919,.396,215,734,.293,372,569,.654,370,1292,343,187,89,436,578,68.5,92.9,.297,.382,.481,51.8,45.1,8.2,8.7,.452,16.6,29.3,.194,2019
3,Alabama A&M,32,5,27,.156,-19.23,-8.38,4,14,4,7,0,18,1938,2285,1295,736,1809,.407,182,578,.315,284,453,.627,314,1032,385,234,50,487,587,67.5,88.7,.250,.320,.479,47.1,52.3,10.7,4.7,.457,19.4,27.6,.157,1295,811,1730,.469,235,665,.353,428,672,.637,333,1158,434,246,109,458,463,67.5,104.5,.388,.384,.558,52.9,53.5,11.3,8.9,.537,18.3,31.7,.247,2019
4,Alabama-Birmingham,35,20,15,.571,0.36,-1.52,10,8,11,5,6,6,2470,2370,1410,906,2003,.452,234,694,.337,424,630,.673,367,1279,401,218,82,399,578,66.6,105.2,.315,.346,.536,52.7,44.3,9.3,7.5,.511,14.8,30.4,.212,1410,811,1960,.414,297,873,.340,451,616,.732,310,1149,444,178,110,421,562,66.6,100.9,.314,.445,.526,47.3,54.7,7.6,8.4,.490,15.7,25.4,.230,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Wright State,35,25,10,.714,0.58,-3.64,14,4,14,2,8,7,2500,2307,1410,873,2019,.432,237,703,.337,517,723,.715,356,1266,460,231,77,459,558,69.9,101.4,.358,.348,.529,51.8,52.7,9.4,6.5,.491,16.3,28.9,.256,1410,816,1965,.415,268,775,.346,407,573,.710,303,1178,420,204,102,531,702,69.9,93.6,.292,.394,.516,48.2,51.5,8.3,7.8,.483,19.2,25.0,.207,2018
347,Wyoming,33,20,13,.606,4.26,3.54,10,8,13,4,5,7,2633,2597,1365,873,1959,.446,304,881,.345,583,790,.738,231,1148,518,215,130,449,654,74.7,103.3,.403,.450,.564,47.1,59.3,8.4,9.8,.523,16.1,19.2,.298,1365,898,2022,.444,260,700,.371,541,752,.719,315,1288,461,203,79,483,708,74.7,101.8,.372,.346,.546,52.9,51.3,8.0,7.3,.508,16.9,25.6,.268,2018
348,Xavier,35,29,6,.829,19.08,9.45,15,3,17,1,9,2,2953,2616,1415,1011,2055,.492,274,736,.372,657,843,.779,345,1342,579,205,109,441,602,72.3,115.5,.410,.358,.601,55.1,57.3,8.0,8.4,.559,15.2,30.2,.320,1415,944,2192,.431,301,902,.334,427,562,.760,296,1095,492,238,127,400,731,72.3,102.3,.256,.411,.532,44.9,52.1,9.3,9.6,.499,14.0,22.9,.195,2018
349,Yale,31,16,15,.516,-4.19,-2.83,9,5,9,3,7,11,2312,2288,1250,831,1797,.462,252,759,.332,398,551,.722,275,1052,507,204,95,407,541,70.1,105.5,.307,.422,.562,49.8,61.0,9.3,8.2,.533,16.5,26.0,.221,1250,824,1793,.460,214,631,.339,426,589,.723,277,1061,428,214,80,397,539,70.1,104.4,.328,.352,.552,50.2,51.9,9.8,7.7,.519,16.1,26.3,.238,2018


In [7]:
#real cell 

#need results for our eventual dataset we want to model on
#bringing in historical tournomant results data, 1985-2017, use season to match years not achademic season
#this line is reading a file of historical results that I had downloaded in the past
results = pd.read_csv('/Users/cmuhlenk/Documents/Current_Projects/misccp/data_science/MarchMadness/mbb_historical_tournament_games.csv')

#getting unique names of teams
rwnames = results[['win_school_ncaa','win_alias','win_name']]
rwnames.rename(columns = {'win_school_ncaa':'rname','win_alias':'alias','win_name':'mascot'}, inplace = True)
rlnames = results[['lose_school_ncaa','lose_alias','lose_name']]
rlnames.rename(columns = {'lose_school_ncaa':'rname','lose_alias':'alias','lose_name':'mascot'}, inplace = True)
rnames = rwnames.append(rlnames) 
rnames = rnames.drop_duplicates()

snames = stats[['School']]
snames = snames.drop_duplicates()

#exporting team names for both results and stats to manually create a mapping
rnames.to_csv('rnames.csv', index = False)
snames.to_csv('snames.csv', index = False)

/Users/cmuhlenk/Documents/Current_Projects/misccp/data_science/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [8]:
#real cell

#some manual mapping needs to be done is excel in order to combine the data
#importing the manually mapped data
mapnames = pd.read_csv (r'mapnames.csv')
mapnames = mapnames[['rname','sname']]

#cutting down results data
results = results[['season','round','win_seed','win_school_ncaa','lose_seed','lose_school_ncaa']]
results.columns =  ['__' + str(col) for col in results.columns]

#breaking stats to win and lose stats to make column naming and merging easier, possibly able to do with out making the seperate table but eh
wstats = stats.copy(deep=True)
wstats.columns = ['w_' + str(col) for col in wstats.columns]
lstats = stats.copy(deep=True)
lstats.columns = ['l_' + str(col) for col in lstats.columns]

#joinging the results data with stats data
#adding alternative name columns to wstats, lstats
wstats_map = pd.merge(wstats, mapnames, left_on='w_School', right_on='sname', how = "inner")
lstats_map = pd.merge(lstats, mapnames, left_on='l_School', right_on='sname', how = "inner")
wstats_map = wstats_map.drop(columns=['sname'])
lstats_map = lstats_map.drop(columns=['sname'])

#merging the results and stats and cleaning data
combined = pd.merge(wstats_map, results, left_on=['rname','w_year'], right_on=['__win_school_ncaa','__season'], how = "inner")
combined = combined.drop(columns=['rname','w_year','__win_school_ncaa'])
combined = pd.merge(lstats_map, combined, left_on=['rname','l_year'], right_on=['__lose_school_ncaa','__season'], how = "inner")
combined = combined.drop(columns=['rname','l_year','__lose_school_ncaa'])
combined = combined.rename(columns={"__win_seed": "w_seed", "__lose_seed": "l_seed"})

#need to fix the dataset so we can have a binary classifier that isnt always the same
#so need to rename winning and loosing team randomly to team one and team two 
#while keeping track of who actually won and how to put everything back together
#this mostliky included unneccessary bits but I had already coded it and didnt want to change things

#adding game ID
combined.insert(0, '__Game_ID', range(0,len(combined)))

#adding split decider and binary classifier
#if split decider is greater than .5 then team 1 won, if not team 2 won
#binary classifier will say if team 1 won or not
np.random.seed(1)
combined['__Split_Decider'] = np.random.randint(0, 99999 , combined.shape[0])
combined['__Split_Decider'] = combined['__Split_Decider']/100000
combined['__t1win'] = np.where(combined['__Split_Decider']>=.5, 1, 0)

#splitting up data so we can rename columns
t1win = combined.loc[combined['__t1win'] == 1]
t1win = t1win.rename(columns={'l_3P':'t2_3P','l_3P%':'t2_3P%','l_3PA':'t2_3PA','l_3PAr':'t2_3PAr','l_AST':'t2_AST','l_AST%':'t2_AST%','l_AwayL':'t2_AwayL','l_AwayW':'t2_AwayW','l_BLK':'t2_BLK','l_BLK%':'t2_BLK%','l_ConfL':'t2_ConfL','l_ConfW':'t2_ConfW','l_FG':'t2_FG','l_FG%':'t2_FG%','l_FGA':'t2_FGA','l_FT':'t2_FT','l_FT%':'t2_FT%','l_FT/FGA':'t2_FT/FGA','l_FTA':'t2_FTA','l_FTr':'t2_FTr','l_G':'t2_G','l_HomeL':'t2_HomeL','l_HomeW':'t2_HomeW','l_L':'t2_L','l_MP':'t2_MP','l_ORB':'t2_ORB','l_ORB%':'t2_ORB%','l_ORtg':'t2_ORtg','l_Opp.':'t2_Opp.','l_PF':'t2_PF','l_Pace':'t2_Pace','l_SOS':'t2_SOS','l_SRS':'t2_SRS','l_STL':'t2_STL','l_STL%':'t2_STL%','l_School':'t2_School','l_TOV':'t2_TOV','l_TOV%':'t2_TOV%','l_TRB':'t2_TRB','l_TRB%':'t2_TRB%','l_TS%':'t2_TS%','l_Tm.':'t2_Tm.','l_W':'t2_W','l_W-L%':'t2_W-L%','l_eFG%':'t2_eFG%','l_opp3P':'t2_opp3P','l_opp3P%':'t2_opp3P%','l_opp3PA':'t2_opp3PA','l_opp3PAr':'t2_opp3PAr','l_oppAST':'t2_oppAST','l_oppAST%':'t2_oppAST%','l_oppBLK':'t2_oppBLK','l_oppBLK%':'t2_oppBLK%','l_oppFG':'t2_oppFG','l_oppFG%':'t2_oppFG%','l_oppFGA':'t2_oppFGA','l_oppFT':'t2_oppFT','l_oppFT%':'t2_oppFT%','l_oppFT/FGA':'t2_oppFT/FGA','l_oppFTA':'t2_oppFTA','l_oppFTr':'t2_oppFTr','l_oppMP':'t2_oppMP','l_oppORB':'t2_oppORB','l_oppORB%':'t2_oppORB%','l_oppORtg':'t2_oppORtg','l_oppPF':'t2_oppPF','l_oppPace':'t2_oppPace','l_oppSTL':'t2_oppSTL','l_oppSTL%':'t2_oppSTL%','l_oppTOV':'t2_oppTOV','l_oppTOV%':'t2_oppTOV%','l_oppTRB':'t2_oppTRB','l_oppTRB%':'t2_oppTRB%','l_oppTS%':'t2_oppTS%','l_oppeFG%':'t2_oppeFG%','w_3P':'t1_3P','w_3P%':'t1_3P%','w_3PA':'t1_3PA','w_3PAr':'t1_3PAr','w_AST':'t1_AST','w_AST%':'t1_AST%','w_AwayL':'t1_AwayL','w_AwayW':'t1_AwayW','w_BLK':'t1_BLK','w_BLK%':'t1_BLK%','w_ConfL':'t1_ConfL','w_ConfW':'t1_ConfW','w_FG':'t1_FG','w_FG%':'t1_FG%','w_FGA':'t1_FGA','w_FT':'t1_FT','w_FT%':'t1_FT%','w_FT/FGA':'t1_FT/FGA','w_FTA':'t1_FTA','w_FTr':'t1_FTr','w_G':'t1_G','w_HomeL':'t1_HomeL','w_HomeW':'t1_HomeW','w_L':'t1_L','w_MP':'t1_MP','w_ORB':'t1_ORB','w_ORB%':'t1_ORB%','w_ORtg':'t1_ORtg','w_Opp.':'t1_Opp.','w_PF':'t1_PF','w_Pace':'t1_Pace','w_SOS':'t1_SOS','w_SRS':'t1_SRS','w_STL':'t1_STL','w_STL%':'t1_STL%','w_School':'t1_School','w_TOV':'t1_TOV','w_TOV%':'t1_TOV%','w_TRB':'t1_TRB','w_TRB%':'t1_TRB%','w_TS%':'t1_TS%','w_Tm.':'t1_Tm.','w_W':'t1_W','w_W-L%':'t1_W-L%','w_eFG%':'t1_eFG%','w_opp3P':'t1_opp3P','w_opp3P%':'t1_opp3P%','w_opp3PA':'t1_opp3PA','w_opp3PAr':'t1_opp3PAr','w_oppAST':'t1_oppAST','w_oppAST%':'t1_oppAST%','w_oppBLK':'t1_oppBLK','w_oppBLK%':'t1_oppBLK%','w_oppFG':'t1_oppFG','w_oppFG%':'t1_oppFG%','w_oppFGA':'t1_oppFGA','w_oppFT':'t1_oppFT','w_oppFT%':'t1_oppFT%','w_oppFT/FGA':'t1_oppFT/FGA','w_oppFTA':'t1_oppFTA','w_oppFTr':'t1_oppFTr','w_oppMP':'t1_oppMP','w_oppORB':'t1_oppORB','w_oppORB%':'t1_oppORB%','w_oppORtg':'t1_oppORtg','w_oppPF':'t1_oppPF','w_oppPace':'t1_oppPace','w_oppSTL':'t1_oppSTL','w_oppSTL%':'t1_oppSTL%','w_oppTOV':'t1_oppTOV','w_oppTOV%':'t1_oppTOV%','w_oppTRB':'t1_oppTRB','w_oppTRB%':'t1_oppTRB%','w_oppTS%':'t1_oppTS%','w_oppeFG%':'t1_oppeFG%','w_seed':'t1_seed','l_seed':'t2_seed'})
t2win = combined.loc[combined['__t1win'] == 0]
t2win = t2win.rename(columns={'l_3P':'t1_3P','l_3P%':'t1_3P%','l_3PA':'t1_3PA','l_3PAr':'t1_3PAr','l_AST':'t1_AST','l_AST%':'t1_AST%','l_AwayL':'t1_AwayL','l_AwayW':'t1_AwayW','l_BLK':'t1_BLK','l_BLK%':'t1_BLK%','l_ConfL':'t1_ConfL','l_ConfW':'t1_ConfW','l_FG':'t1_FG','l_FG%':'t1_FG%','l_FGA':'t1_FGA','l_FT':'t1_FT','l_FT%':'t1_FT%','l_FT/FGA':'t1_FT/FGA','l_FTA':'t1_FTA','l_FTr':'t1_FTr','l_G':'t1_G','l_HomeL':'t1_HomeL','l_HomeW':'t1_HomeW','l_L':'t1_L','l_MP':'t1_MP','l_ORB':'t1_ORB','l_ORB%':'t1_ORB%','l_ORtg':'t1_ORtg','l_Opp.':'t1_Opp.','l_PF':'t1_PF','l_Pace':'t1_Pace','l_SOS':'t1_SOS','l_SRS':'t1_SRS','l_STL':'t1_STL','l_STL%':'t1_STL%','l_School':'t1_School','l_TOV':'t1_TOV','l_TOV%':'t1_TOV%','l_TRB':'t1_TRB','l_TRB%':'t1_TRB%','l_TS%':'t1_TS%','l_Tm.':'t1_Tm.','l_W':'t1_W','l_W-L%':'t1_W-L%','l_eFG%':'t1_eFG%','l_opp3P':'t1_opp3P','l_opp3P%':'t1_opp3P%','l_opp3PA':'t1_opp3PA','l_opp3PAr':'t1_opp3PAr','l_oppAST':'t1_oppAST','l_oppAST%':'t1_oppAST%','l_oppBLK':'t1_oppBLK','l_oppBLK%':'t1_oppBLK%','l_oppFG':'t1_oppFG','l_oppFG%':'t1_oppFG%','l_oppFGA':'t1_oppFGA','l_oppFT':'t1_oppFT','l_oppFT%':'t1_oppFT%','l_oppFT/FGA':'t1_oppFT/FGA','l_oppFTA':'t1_oppFTA','l_oppFTr':'t1_oppFTr','l_oppMP':'t1_oppMP','l_oppORB':'t1_oppORB','l_oppORB%':'t1_oppORB%','l_oppORtg':'t1_oppORtg','l_oppPF':'t1_oppPF','l_oppPace':'t1_oppPace','l_oppSTL':'t1_oppSTL','l_oppSTL%':'t1_oppSTL%','l_oppTOV':'t1_oppTOV','l_oppTOV%':'t1_oppTOV%','l_oppTRB':'t1_oppTRB','l_oppTRB%':'t1_oppTRB%','l_oppTS%':'t1_oppTS%','l_oppeFG%':'t1_oppeFG%','w_3P':'t2_3P','w_3P%':'t2_3P%','w_3PA':'t2_3PA','w_3PAr':'t2_3PAr','w_AST':'t2_AST','w_AST%':'t2_AST%','w_AwayL':'t2_AwayL','w_AwayW':'t2_AwayW','w_BLK':'t2_BLK','w_BLK%':'t2_BLK%','w_ConfL':'t2_ConfL','w_ConfW':'t2_ConfW','w_FG':'t2_FG','w_FG%':'t2_FG%','w_FGA':'t2_FGA','w_FT':'t2_FT','w_FT%':'t2_FT%','w_FT/FGA':'t2_FT/FGA','w_FTA':'t2_FTA','w_FTr':'t2_FTr','w_G':'t2_G','w_HomeL':'t2_HomeL','w_HomeW':'t2_HomeW','w_L':'t2_L','w_MP':'t2_MP','w_ORB':'t2_ORB','w_ORB%':'t2_ORB%','w_ORtg':'t2_ORtg','w_Opp.':'t2_Opp.','w_PF':'t2_PF','w_Pace':'t2_Pace','w_SOS':'t2_SOS','w_SRS':'t2_SRS','w_STL':'t2_STL','w_STL%':'t2_STL%','w_School':'t2_School','w_TOV':'t2_TOV','w_TOV%':'t2_TOV%','w_TRB':'t2_TRB','w_TRB%':'t2_TRB%','w_TS%':'t2_TS%','w_Tm.':'t2_Tm.','w_W':'t2_W','w_W-L%':'t2_W-L%','w_eFG%':'t2_eFG%','w_opp3P':'t2_opp3P','w_opp3P%':'t2_opp3P%','w_opp3PA':'t2_opp3PA','w_opp3PAr':'t2_opp3PAr','w_oppAST':'t2_oppAST','w_oppAST%':'t2_oppAST%','w_oppBLK':'t2_oppBLK','w_oppBLK%':'t2_oppBLK%','w_oppFG':'t2_oppFG','w_oppFG%':'t2_oppFG%','w_oppFGA':'t2_oppFGA','w_oppFT':'t2_oppFT','w_oppFT%':'t2_oppFT%','w_oppFT/FGA':'t2_oppFT/FGA','w_oppFTA':'t2_oppFTA','w_oppFTr':'t2_oppFTr','w_oppMP':'t2_oppMP','w_oppORB':'t2_oppORB','w_oppORB%':'t2_oppORB%','w_oppORtg':'t2_oppORtg','w_oppPF':'t2_oppPF','w_oppPace':'t2_oppPace','w_oppSTL':'t2_oppSTL','w_oppSTL%':'t2_oppSTL%','w_oppTOV':'t2_oppTOV','w_oppTOV%':'t2_oppTOV%','w_oppTRB':'t2_oppTRB','w_oppTRB%':'t2_oppTRB%','w_oppTS%':'t2_oppTS%','w_oppeFG%':'t2_oppeFG%','w_seed':'t2_seed','l_seed':'t1_seed'})
recombined = t1win.append(t2win) 

#get rid of unneccesarry columns
recombined = recombined.drop(columns=['__Game_ID','__Split_Decider','__season','t1_School','t2_School'])

#replace columns with blank spaces with missing values
recombined = recombined.replace('', np.NaN)

In [3]:
#real cell

#some of these will likely need you to download the correct module in terminal, for example happened with fancy impute
#needed to type:
#  pip install fancyimpute
#this was also throwing errors, needed to install x-code, see here https://ma.ttias.be/mac-os-xcrun-error-invalid-active-developer-path-missing-xcrun/

#importing all necessary libraries for model creation
import scipy
import sklearn
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from fancyimpute import KNN
from sklearn.metrics import roc_auc_score

In [9]:
#real cell

#sklearn classifiers don't accept missing vallues so we need to replace them, use fancy imputer to do this so we can impute based on knn
# Use 3 nearest rows which have a feature to fill in each row's missing features
#this line looks more complicated because fancy imputer returens a array and we want a data frame with the correct column names
recombined_knn = pd.DataFrame(np.round(KNN(k=3).fit_transform(recombined)),columns = recombined.columns)

#now that we have the data in a workable form we can create the model

#dividinging data into features and lables
#selecting the features
X = recombined_knn.loc[:, recombined_knn.columns != '__t1win'].values
# Now let's tell the dataframe which column we want for the target/labels.  
y = recombined_knn['__t1win']

#creating test and training sets
# Test size specifies how much of the data you want to set aside for the testing set. 
# Random_state parameter is just a random seed we can use.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=27)

Imputing row 1/1613 with 68 missing, elapsed time: 1.551
Imputing row 101/1613 with 69 missing, elapsed time: 1.635
Imputing row 201/1613 with 72 missing, elapsed time: 1.725
Imputing row 301/1613 with 68 missing, elapsed time: 1.811
Imputing row 401/1613 with 80 missing, elapsed time: 1.886
Imputing row 501/1613 with 72 missing, elapsed time: 1.960
Imputing row 601/1613 with 69 missing, elapsed time: 2.048
Imputing row 701/1613 with 0 missing, elapsed time: 2.133
Imputing row 801/1613 with 0 missing, elapsed time: 2.205
Imputing row 901/1613 with 68 missing, elapsed time: 2.286
Imputing row 1001/1613 with 72 missing, elapsed time: 2.362
Imputing row 1101/1613 with 0 missing, elapsed time: 2.438
Imputing row 1201/1613 with 72 missing, elapsed time: 2.522
Imputing row 1301/1613 with 0 missing, elapsed time: 2.585
Imputing row 1401/1613 with 68 missing, elapsed time: 2.664
Imputing row 1501/1613 with 68 missing, elapsed time: 2.754
Imputing row 1601/1613 with 72 missing, elapsed time: 2.

In [10]:
#real cell

#we are creating a loop to fit various models listed under names/classifiers
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, probability = True),
    SVC(gamma=2, C=1, probability = True),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

#this is the table that will keep track of the records of how each model preforms
records = pd.DataFrame(columns=['name', 'score', 'AUC'])

for name, clf in zip(names, classifiers):
    model = clf
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    auc = roc_auc_score(y_test, probs)
    records = records.append({'name': name, 'score': score, 'AUC': auc}, ignore_index=True)

records

/Users/cmuhlenk/Documents/Current_Projects/misccp/data_science/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


,name,score,AUC
0,Nearest Neighbors,0.733471,0.764077
1,Linear SVM,0.820248,0.914812
2,RBF SVM,0.516529,0.500000
3,Gaussian Process,0.516529,0.500000
4,Decision Tree,0.766529,0.828120
5,Random Forest,0.776860,0.857983
6,Neural Net,0.619835,0.802462
7,AdaBoost,0.793388,0.901726
8,Naive Bayes,0.789256,0.870342
9,QDA,0.673554,0.717111


In [11]:
#real call

#looking at the records table we see that the linear SVM model preforms the best, i.e. higher relative numbers
#so we create the model based on that
model = SVC(kernel="linear", C=0.025, probability = True)
model.fit(X_train, y_train)


SVC(C=0.025, kernel='linear', probability=True)

In [12]:
#real cell, in progress for testing, date is the wrong date, need to replace

#now we need the data from this year for predictions, this is just mostly repeating earlier steps with new data and slightly different names

year = 2021
url = "https://www.sports-reference.com/cbb/seasons/"+ str(year) +"-school-stats.html"
html = urlopen(url)
soup = BeautifulSoup(html)

soup.findAll('tr', limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
headers = headers[1:]

length = len(headers)
for i in range(length):
    if i == 8 or i == 9:
        headers[i] = 'Conf'+headers[i]
    elif i == 11 or i == 12:
        headers[i] = 'Home'+headers[i]
    elif i == 14 or i == 15:
        headers[i] = 'Away'+headers[i]

rows = soup.findAll('tr')[2:]
currentpage_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

current_stats = pd.DataFrame(currentpage_stats, columns = headers)

current_stats = current_stats.dropna(subset= ['School'])
current_stats = current_stats.drop(['\xa0'], axis=1)

TableType = ['-advanced-school', '-opponent', '-advanced-opponent']

for tt in TableType:
    url = "https://www.sports-reference.com/cbb/seasons/"+ str(year) + tt +"-stats.html"
    html = urlopen(url)
    soup = BeautifulSoup(html)
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    headers = headers[1:]
    if tt == '-opponent' or tt == '-advanced-opponent':
        j = 0
        for i in headers:
            if i !=  'School':
                headers[j] = 'opp'+ i
            j = j + 1    
    
    rows = soup.findAll('tr')[2:]
    currentpage_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    Astats = pd.DataFrame(currentpage_stats, columns = headers)

    if tt == '-opponent' or tt == '-advanced-opponent':
        Astats = Astats.dropna(subset= ['School'])
        Astats = Astats.drop(['oppG','oppW','oppL','oppW-L%','oppSRS','oppSOS','oppW','oppL','oppW','oppL','oppW','oppL','oppTm.','oppOpp.','opp\xa0'], axis=1)
    else:
        Astats = Astats.dropna(subset= ['School'])
        Astats = Astats.drop(['G','W','L','W-L%','SRS','SOS','W','L','W','L','W','L','Tm.','Opp.','\xa0'], axis=1)
        
    current_stats = current_stats.merge(Astats, how = 'outer', on = 'School')

current_stats['School'] = current_stats['School'].map(lambda x: x.rstrip('\xa0NCAA'))    

In [35]:
%%capture [--no-stderr]
#this is to prevent a warning about copying over data which is what we are trying to do

#real cell

#need to bring in data for match ups

#Bracket data set comes form manually entering teams being considered
#data has info on teams playing, their seeds, a gameid, what gameid, if they win, and how they progress
bracket = pd.read_csv('/Users/cmuhlenk/Documents/Current_Projects/misccp/data_science/MarchMadness/Bracket.csv')

#itterate through the entire bracket 
for i in range(63):
    #grab the data for a single game
    matchup = bracket[i:i+1]
    matchup = matchup.drop(columns=['GameId','winner','toID','toSpot'])

    #creat copies of current_stats to make merging easy
    t1_stats = current_stats.copy(deep=True)
    t1_stats.columns = ['t1_' + str(col) for col in t1_stats.columns]
    t2_stats = current_stats.copy(deep=True)
    t2_stats.columns = ['t2_' + str(col) for col in t2_stats.columns]

    #merge data on school names
    current_combined = pd.merge(t1_stats, matchup, left_on=['t1_School'], right_on=['t1n'], how = "inner")
    current_combined = current_combined.drop(columns=['t1n'])
    current_combined = pd.merge(t2_stats, current_combined, left_on=['t2_School'], right_on=['t2n'], how = "inner")
    current_combined = current_combined.drop(columns=['t2n', 't1_School','t2_School'])

    #sort data so it's in the same form as the model
    current_combined = current_combined.reindex(sorted(current_combined.columns), axis=1)
    
    #apply model to get prediction
    prediction = model.predict(current_combined)

    #changing table to reflect results of the prediction
    bracket['winner'][i] = prediction
    toID = bracket['toID'][i]-1
    tospot = bracket['toSpot'][i]
    bracket['winner'][i] = prediction
    if toID != 999:
        if prediction == 1:
            if tospot == 1:
                bracket['t1n'][toID] = bracket['t1n'][i]
                bracket['t1_seed'][toID] = bracket['t1_seed'][i]
            elif tospot == 2:
                bracket['t2n'][toID] = bracket['t1n'][i]
                bracket['t2_seed'][toID] = bracket['t1_seed'][i]
        elif prediction == 0:
            if tospot == 1:
                bracket['t1n'][toID] = bracket['t2n'][i]
                bracket['t1_seed'][toID] = bracket['t2_seed'][i]
            elif tospot == 2:
                bracket['t2n'][toID] = bracket['t2n'][i]
                bracket['t2_seed'][toID] = bracket['t2_seed'][i]    
    elif toID == 999:
        break


In [30]:
#test cell

#current_combined

#t2_stats



In [36]:
#real cell

#the output of the results, a winner of 1 means team 1 a.k.a t1n wins, and 0 means team 2 wins
bracket

,GameId,round,t1n,t2n,t1_seed,t2_seed,winner,toID,toSpot
0,1,64,Gonzaga,Appalachian State,1.0,16.0,1.0,33,1
1,2,64,Oklahoma,Missouri,8.0,9.0,1.0,33,2
2,3,64,Creighton,UC-Santa Barbara,5.0,12.0,1.0,34,1
3,4,64,Virginia,Ohio,4.0,13.0,1.0,34,2
4,5,64,Southern California,Wichita State,6.0,11.0,1.0,35,1
5,6,64,Kansas,Eastern Washington,3.0,14.0,1.0,35,2
6,7,64,Oregon,Virginia Commonwealth,7.0,10.0,1.0,36,1
7,8,64,Iowa,Grand Canyon,2.0,15.0,0.0,36,2
8,9,64,Michigan,Mount St. Mary's,1.0,16.0,1.0,37,1
9,10,64,Louisiana State,St. Bonaventure,8.0,9.0,1.0,37,2


In [34]:
# when testing with historical data the predictions made sense. However using the actual data from the 2021 teams, teams with good records from small colleges where predicted to do much better than expected. 
# While this is likely partially due to complications in the seanon due to the covid pandemic, the model needs further tweeking, ideally seeding and strength of schedule metrics should provide higher predicitive power than they did.
# Further work is necessary, looking forward to 2022.


array([1.])